In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium_stealth import stealth
import time

options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
# options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

service = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(options=options, service=service)

stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )


In [7]:
daraz=driver.get("https://www.topuniversities.com/world-university-rankings")  # open website
print(driver.current_url)
print(driver.title)

https://www.topuniversities.com/world-university-rankings
QS World University Rankings 2026: Top Global Universities | Top Universities


In [9]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time



time.sleep(3)  # extra wait for dynamic content

try:
    button=driver.find_element(By.CLASS_NAME,"eu-cookie-compliance-default-button eu-cookie-compliance-reject-button button button--small")
    button.click()
except:
    print("No such elment")
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.ID, "ranking-data-load"))
)

rows = driver.find_elements(By.CLASS_NAME, "ind")[:50]

data = []
for row in rows:
    try:
        university = row.find_element(By.CLASS_NAME, "uni-link").text.strip()
    except:
        university = None

    try:
        country = row.find_element(By.CLASS_NAME, "location").text.strip()
    except:
        country = None

    try:
        score = row.find_element(By.CSS_SELECTOR, ".rank-score").text.strip()
    except:
        score = None


    data.append({
        "University": university,
        "Country": country,
        "Overall Score": score,
    })


driver.quit()

df = pd.DataFrame(data)
df.to_csv("top_universities.csv",index=False)




No such elment


In [10]:

df=pd.read_csv("top_universities.csv")
df["Overall Score"] = pd.to_numeric(df["Overall Score"], errors="coerce")

print(df.head())


country_counts = df["Country"].value_counts()

# Average scores per country
country_scores = df.groupby("Country")["Overall Score"].mean().sort_values(ascending=False)

print("\n Number of Universities per Country:")
print(country_counts)

print("\n Average Overall Score by Country:")
print(country_scores)

                                    University                   Country  \
0  Massachusetts Institute of Technology (MIT)  Cambridge, United States   
1                      Imperial College London    London, United Kingdom   
2                          Stanford University   Stanford, United States   
3                         University of Oxford    Oxford, United Kingdom   
4                           Harvard University  Cambridge, United States   

   Overall Score  
0          100.0  
1           99.4  
2           98.9  
3           97.9  
4           97.7  

 Number of Universities per Country:
Country
Cambridge, United States       2
London, United Kingdom         2
Singapore, Singapore           2
Sydney, Australia              2
Beijing, China (Mainland)      2
Cambridge, United Kingdom      1
Zürich, Switzerland            1
Oxford, United Kingdom         1
Stanford, United States        1
Hong Kong, Hong Kong SAR       1
Pasadena, United States        1
Philadelphia, United